# Homework 2: Word Similarity

Student Name: Yu Dong

Student ID: 928922

## General info

<b>Due date</b>: Thursday, 18 Apr 2019 4pm

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 7% of mark for class (with 6% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib and Scikit-Learn. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>. 

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dateset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

Instructions: For this homework we will be comparing our methods against a popular dataset of word similarities called Similarity-353. You need to first obtain this dataset, which is available on LMS. The file we will be using is called set1.tab. Make sure you save this in the same folder as the notebook. Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. You should ignore the subsequent columns.
Here shows the first six lines of the file:
Word 1    Word 2    Human (mean)    1    2    3    4    5    6    7    8    9    10    11    12    13    
love    sex    6.77    9    6    8    8    7    8    8    4    7    2    6    7    8    
tiger    cat    7.35    9    7    8    7    8    9    8.5    5    6    9    7    5    7    
tiger    tiger    10.00    10    10    10    10    10    10    10    10    10    10    10    10    10    
book    paper    7.46    8    8    7    7    8    9    7    6    7    8    9    4    9    
computer    keyboard    7.62    8    7    9    9    8    8    7    7    6    8    10    3    9
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.
The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this homework, we will be treating the paragraphs of the Brown corpus as our "documents". You can iterate over them by using the paras method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized. Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of less than 8 in this corpus.
For this part, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called filtered_gold_standard. You may check the section, "For your testing", below for the expected filtered_gold_standard.
(1 mark)

In [1]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet
import pandas as pd
import re

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

###
# Your answer BEGINS HERE
###
# build a dictionary to store ('Word1', 'Word2') as key and similarity score as value in 'set1.tab'
f = open("set1.tab", 'r')
f.readline()
build_dict = {}
for row in f:
    column = row.split()
    build_dict[(column[0], column[1])] = float(column[2])
#a function to lemmatize word
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word, 'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word, 'n')
    return lemma
#process paragraph in brown, use a set to store the words in each paragraph.(for document frequency calculation)
build_list = []
for paragraph in brown.paras():
    word_set = set()
    for sentence in paragraph:
        for word in sentence:
            if re.match(r'[a-zA-Z]+',word) is not None:
                lemma = lemmatizer.lemmatize(word.lower())
                word_set.add(lemma)
    build_list.append(word_set)
#a function to calculate the document frequency of each word type                
def word_freq(word, alist):
    count = 0
    for words in alist:
        if word in words:
            count += 1
    return count
#store the word pair to dictionary if the document frequency of two words are both more than eight
for wordpair, similarity in build_dict.items():
    if word_freq(wordpair[0], build_list) >= 8 and word_freq(wordpair[1], build_list) >= 8:
        filtered_gold_standard[wordpair] = similarity
###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)

[nltk_data] Downloading package brown to /Users/yunadong/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yunadong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
94
{('bank', 'money'): 8.12, ('tiger', 'cat'): 7.35, ('tool', 'implement'): 6.46, ('money', 'wealth'): 8.27, ('journey', 'voyage'): 9.29, ('cup', 'food'): 5.0, ('train', 'car'): 6.31, ('coast', 'shore'): 9.1, ('psychology', 'anxiety'): 7.0, ('money', 'cash'): 9.08, ('journey', 'car'): 5.85, ('money', 'deposit'): 7.73, ('book', 'paper'): 7.46, ('money', 'dollar'): 8.42, ('wood', 'forest'): 7.73, ('precedent', 'information'): 3.85, ('cup', 'object'): 3.69, ('plane', 'car'): 5.77, ('money', 'bank'): 8.5, ('tiger', 'tiger'): 10.0, ('drink', 'mother'): 2.65, ('tiger', 'animal'): 7.0, ('cup', 'liquid'): 5.9, ('football', 'basketball'): 6.81, ('coast', 'hill'): 4.38, ('drink', 'mouth'): 5.96, ('monk', 'slave'): 0.92, ('doctor', 'nurse'

<b>For your testing: </b>

In [2]:
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)

In [3]:
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either a) having only one sense (i.e. only one synset), or b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria.

When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*. You may check the section, <i>"For your testing"</i>, below for the expected *final_gold_standard*.

(1 mark)

In [4]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}

###
# Your answer BEGINS HERE
###
#create a dictionary to store word primary sense for lin_similarity caculation
primary_sense = {}
#two condition for single primary sense
def single_primary_sense(word):
    #first condition: only have one sense and the primary sense is a noun
    if len(wordnet.synsets(word)) ==1 and '.n.' in wordnet.synsets(word)[0].name():
        primary_sense[word] = wordnet.synsets(word)[0].name()
        return True
    findcount = []
    for synset in wordnet.synsets(word):
        count = 0
        for lemma in synset.lemmas():
            if lemma.name() == word :
                count+=lemma.count()
        findcount.append(count)
    findcount.sort(reverse=True) 
    #second condition: most common sense is at least 4 times larger than the second common sense and primary sense is noun
    if findcount[0]>=4*findcount[1] and '.n.' in wordnet.synsets(word)[0].name():
        primary_sense[word] = wordnet.synsets(word)[findcount.index(max(findcount))].name()
        return True
    return False
#store the wordpair with its score to final dicitonary, which meets the condition above
for wordpair, similarity in filtered_gold_standard.items():
    if single_primary_sense(wordpair[0]) and single_primary_sense(wordpair[1]):
        final_gold_standard[wordpair] = similarity
###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)

27
{('football', 'basketball'): 6.81, ('coast', 'hill'): 4.38, ('forest', 'graveyard'): 1.85, ('furnace', 'stove'): 8.79, ('psychology', 'mind'): 7.69, ('journey', 'voyage'): 9.29, ('student', 'professor'): 6.81, ('brother', 'monk'): 6.27, ('coast', 'forest'): 3.15, ('coast', 'shore'): 9.1, ('baby', 'mother'): 7.85, ('planet', 'galaxy'): 8.11, ('bread', 'butter'): 6.19, ('money', 'cash'): 9.08, ('journey', 'car'): 5.85, ('monk', 'slave'): 0.92, ('bishop', 'rabbi'): 6.69, ('psychology', 'health'): 7.23, ('car', 'automobile'): 8.94, ('alcohol', 'chemistry'): 5.54, ('stock', 'egg'): 1.81, ('psychology', 'science'): 6.71, ('planet', 'moon'): 8.08, ('psychology', 'doctor'): 6.42, ('football', 'tennis'): 6.63, ('professor', 'doctor'): 6.62, ('king', 'queen'): 8.58}


<b>For your testing:</b>

In [5]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)

In [6]:
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Word similiarity scores with Lin similarity, NPMI and LSA (3 marks)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *lin_similarities*. 

(1 mark)

In [7]:
from nltk.corpus import wordnet_ic
from nltk.corpus import wordnet as wn
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###
for key in final_gold_standard.keys():
    semcor_ic = wordnet_ic.ic('ic-semcor.dat')
    lin_similarities[key] = wn.synset(primary_sense[key[0]]).lin_similarity(wn.synset(primary_sense[key[1]]),semcor_ic)
###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /Users/yunadong/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!
{('football', 'basketball'): 0.7704674954719576, ('coast', 'hill'): 0.7285879249527557, ('forest', 'graveyard'): -0.0, ('psychology', 'science'): 0.8124859588519519, ('psychology', 'mind'): 0.3301492601041217, ('journey', 'voyage'): 0.8276818213211407, ('student', 'professor'): 0.23953150362674094, ('brother', 'monk'): 0.20788439871426262, ('coast', 'shore'): 0.9632037579820005, ('baby', 'mother'): 0.654948986686927, ('planet', 'galaxy'): -0.0, ('bread', 'butter'): 0.726992234769291, ('money', 'cash'): 0.8137879336398085, ('journey', 'car'): -0.0, ('monk', 'slave'): 0.20112728495581914, ('bishop', 'rabbi'): 0.7522246770216547, ('psychology', 'health'): 0.08338132296510331, ('car', 'automobile'): 1.0, ('alcohol', 'chemistry'): 0.09135314199193188, ('stock', 'egg'): -0.0, ('professor', 'doctor'): 0.7770369763910114, ('furnace', 'stove'): 0.229

<b>For your testing:</b>

In [8]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-\log_2(p(x,y))} = \frac{\log_2(p(x)p(y))}{\log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NMPI is -1. NPMI is normalized between [-1, +1].

You should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *NPMI_similarities*. 

(1 mark)

In [9]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###
import math
#numbers of paragraphs with the occurance of word1 as word1_count,
#numbers of paragraphs with the occurance of word2 as word2_count,
#numbers of paragraphs with the occurance of both word1 and word2 as word1_count,
#the total number of word types as total_count
#use the NPMI formula above to calculate similarity between word1 and word2
def get_NPMI(word1,word2):
    word1_count = 0
    word2_count = 0
    both_count = 0
    total_count = 0.0
    for word in build_list:
        total_count += 1
        if word1 in word:
            word1_count += 1
            if word2 in word:
                both_count += 1
        if word2 in word:
            word2_count += 1
    if both_count == 0:
        return -1
    return math.log((word1_count/total_count)*(word2_count/total_count),2)/(math.log((both_count/total_count),2))-1

for key in final_gold_standard.keys():
    NPMI_similarities[key] = get_NPMI(key[0],key[1])

###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('football', 'basketball'): 0.5915628706621843, ('coast', 'hill'): 0.08704860269296755, ('forest', 'graveyard'): -1, ('psychology', 'science'): 0.4111983994246273, ('psychology', 'mind'): 0.2148882532394769, ('journey', 'voyage'): -1, ('student', 'professor'): 0.301466577187927, ('brother', 'monk'): 0.208049938966947, ('coast', 'shore'): 0.3987378908811283, ('baby', 'mother'): 0.27919009073497625, ('planet', 'galaxy'): -1, ('bread', 'butter'): 0.5007913913288526, ('money', 'cash'): 0.2046594723539341, ('journey', 'car'): -1, ('monk', 'slave'): -1, ('bishop', 'rabbi'): -1, ('psychology', 'health'): -1, ('car', 'automobile'): 0.3095043070280723, ('alcohol', 'chemistry'): 0.4785391888116599, ('stock', 'egg'): 0.1304209593405543, ('professor', 'doctor'): -1, ('furnace', 'stove'): -1, ('planet', 'moon'): 0.4982739016302409, ('psychology', 'doctor'): 0.25564861465213085, ('football', 'tennis'): -1, ('coast', 'forest'): 0.23582555901568392, ('king', 'queen'): 0.19166947549598023}


<b>For your testing:</b>

In [10]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

**Instructions:** As PMI matrix is very sparse and can be approximated well by a dense representation via singular value decomposition (SVD), you will derive similarity scores using the Latent Semantic Analysis (LSA) method, i.e. apply SVD and truncate to get a dense vector representation of a word type and then calculate cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for your word pairs. 

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*. You may check the section, <i>"For your testing"</i>, below for the expected *LSA_similarities*. 

(1 mark)

In [11]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###
from sklearn.feature_extraction import DictVectorizer
def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word.lower()] = BOW.get(word,0)+1
    return BOW
texts = []
for word in build_list:
    texts.append(get_BOW(word))
#build vectors    
vectorizer = DictVectorizer()
brown_matrix = vectorizer.fit_transform(texts)
#use truncated SVD to get dense vector, get a marix with 500 dimensions
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = 500)
brown_matrix = svd.fit_transform(brown_matrix.transpose())
vec_feature=vectorizer.get_feature_names()
#measure the similarity between two word vectors
from scipy.spatial.distance import cosine as cos_distance
for key in final_gold_standard.keys():
    LSA_similarities[key] = 1- cos_distance(brown_matrix[vec_feature.index(key[0])],brown_matrix[vec_feature.index(key[1])])
###
# Your answer ENDS HERE
###

print(LSA_similarities)

{('football', 'basketball'): 0.22793637425789626, ('coast', 'hill'): 0.17350553881679387, ('forest', 'graveyard'): 0.034718970067624144, ('psychology', 'science'): 0.25665022519737368, ('psychology', 'mind'): 0.11743239598396582, ('journey', 'voyage'): 0.11217020766676233, ('student', 'professor'): 0.27935056833212579, ('brother', 'monk'): 0.077736643340786671, ('coast', 'shore'): 0.3627061885886973, ('baby', 'mother'): 0.32646593818112746, ('planet', 'galaxy'): 0.05642857862550732, ('bread', 'butter'): 0.2765227777517103, ('money', 'cash'): 0.14672891011750699, ('journey', 'car'): -0.0093953902101722164, ('monk', 'slave'): -0.038938814616950701, ('bishop', 'rabbi'): 0.026758985367883015, ('psychology', 'health'): 0.0027104985265405945, ('car', 'automobile'): 0.33059426337217657, ('alcohol', 'chemistry'): 0.090911661535535759, ('stock', 'egg'): 0.12870115641971158, ('professor', 'doctor'): 0.074089034306379808, ('furnace', 'stove'): 0.11891808623628242, ('planet', 'moon'): 0.3810460095

<b>For your testing:</b>

In [12]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## 3. Comparison with the Gold Standard (1 mark)


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA'). You may check the section, <i>"For your testing"</i>, below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(1 mark)


In [13]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###
similarity = []
similarity1 = []
#pearson_correlations between gold standard similarity and 'lin' similarity
for key, value in final_gold_standard.items():
    for key1,value1 in lin_similarities.items():
        if key == key1:
            similarity.append(value)
            similarity1.append(value1)
            pearson_correlations['lin'] = pearsonr(similarity,similarity1)[0]
#person_correlations between gold standard similarity and 'NPMI' similarity            
for key, value in final_gold_standard.items():
    for key1,value1 in NPMI_similarities.items():
        if key == key1:
            similarity.append(value)
            similarity1.append(value1)
            pearson_correlations['NPMI'] = pearsonr(similarity,similarity1)[0]
#person_correlations between gold standard similarity and 'LSA' similarity   
for key, value in final_gold_standard.items():
    for key1,value1 in LSA_similarities.items():
        if key == key1:
            similarity.append(value)
            similarity1.append(value1)
            pearson_correlations['LSA'] = pearsonr(similarity,similarity1)[0]
###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'lin': 0.49414407251029874, 'LSA': 0.20837052797845659, 'NPMI': 0.21697551196972964}


/Users/yunadong/Library/Enthought/Canopy/edm/envs/User/lib/python3.5/site-packages/scipy/stats/stats.py:3003: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


<b>For your testing:</b>

In [14]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## Challenge yourself: Improving the correlation (This part will NOT be marked)

You can try to derive a similarity score from word2vec vectors, using the Gensim interface, and compare it with the similarity scores you've created and the gold standard. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes).

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.